## Traveling Salesman Problem and GA

Following from: https://towardsdatascience.com/evolution-of-a-salesman-a-complete-genetic-algorithm-tutorial-for-python-6fe5d2b3ca35

In [ ]:
%matplotlib notebook
import numpy as np, random, operator, pandas as pd, matplotlib.pyplot as plt
from collections import Counter

In [ ]:
class City:
    def __init__(self, x, y):
        self.x = x
        self.y = y
    
    def distance(self, city):
        xDis = abs(self.x - city.x)
        yDis = abs(self.y - city.y)
        distance = np.sqrt((xDis ** 2) + (yDis ** 2))
        return distance
    
    def __repr__(self):
        return "(" + str(self.x) + "," + str(self.y) + ")"
    
    def __lt__(self, other):
        if self.x < other.x:
            return True
        elif self.x == other.x and self.y < other.y:
            return True
        else:
            return False

In [ ]:
class Fitness:
    def __init__(self, route):
        self.route = route
        self.distance = 0
        self.fitness= 0.0
    
    def routeDistance(self):
        if self.distance ==0:
            pathDistance = 0
            for i in range(0, len(self.route)):
                fromCity = self.route[i]
                toCity = None
                if i + 1 < len(self.route):
                    toCity = self.route[i + 1]
                else:
                    toCity = self.route[0]
                pathDistance += fromCity.distance(toCity)
            self.distance = pathDistance
        return self.distance
    
    def routeFitness(self):
        if self.fitness == 0:
            self.fitness = 1 / float(self.routeDistance())
        return self.fitness

In [ ]:
def createRoute(cityList):
    route = random.sample(cityList, len(cityList))
    return route

In [ ]:
def initialPopulation(popSize, cityList):
    population = []

    for i in range(0, popSize):
        population.append(createRoute(cityList))
    return population

In [ ]:
def rankRoutes(population):
    fitnessResults = {}
    for i in range(0,len(population)):
        fitnessResults[i] = Fitness(population[i]).routeFitness()
    return sorted(fitnessResults.items(), key = operator.itemgetter(1), reverse = True)

In [ ]:
def selection(popRanked, eliteSize):
    selectionResults = []
    df = pd.DataFrame(np.array(popRanked), columns=["Index","Fitness"])
    df['cum_sum'] = df.Fitness.cumsum()
    df['cum_perc'] = 100*df.cum_sum/df.Fitness.sum()
    
    for i in range(0, eliteSize):
        selectionResults.append(popRanked[i][0])
    for i in range(0, len(popRanked) - eliteSize):
        pick = 100*random.random()
        for i in range(0, len(popRanked)):
            if pick <= df.iat[i,3]:
                selectionResults.append(popRanked[i][0])
                break
    return selectionResults

In [ ]:
def matingPool(population, selectionResults):
    matingpool = []
    for i in range(0, len(selectionResults)):
        index = selectionResults[i]
        matingpool.append(population[index])
    return matingpool

In [ ]:
def breed(parent1, parent2):
    child = []
    childP1 = []
    childP2 = []
    
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)

    for i in range(startGene, endGene):
        childP1.append(parent1[i])
        
    childP2 = [item for item in parent2 if item not in childP1]

    child = childP1 + childP2
    return child

In [ ]:
#Partially Mapped Crossover method of recombination/child generation
def PMX(parent1, parent2):
    child = [None] * len(parent1)
    
    #Choose 2 crossover points at random
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)
    
    tmpIndex = 0
    i = None #Holds the gene in parent2 that needs placed
    j = None #Holds the gene in parent1 that is in the place i was in parent 2
    k = None #Used to find a position to place i in the child
    
    for index in range(startGene, endGene):
        child[index] = parent1[index]
        
        if parent2[index] not in parent1[startGene:endGene]:
            i = parent2[index]
            j = parent1[index]
            tmpIndex = parent2.index(j)
            
            while startGene <= tmpIndex and tmpIndex < endGene:
                k = parent1[tmpIndex]
                tmpIndex = parent2.index(k)

            child[tmpIndex] = i
            
    for index in range(0, len(parent2)):
        if child[index] == None:
            child[index] = parent2[index]
                
    return child


In [ ]:
def CalcIndex(lst, cIndex):
    
    lIndex = cIndex - 1
    rIndex = cIndex + 1
        
    if lIndex < 0:
        lIndex = len(lst) - 1
            
    if rIndex >= len(lst):
        rIndex = 0
        
    dictry = {"left":lIndex, "right":rIndex}
        
    return dictry

In [ ]:
def GenerateEdgeTable(parent1, parent2):
    edges = {x: [] for x in parent1}
    
    cIndex = None
    sideIn = None
    
    for index in range(0, len(parent1)):
        sideIn = CalcIndex(parent1, index)
        
        #print(f"cIndex: {index}, lIndex: {sideIn['left']}, rIndex: {sideIn['right']}\n")
        
        edges[parent1[index]].append(parent1[sideIn["left"]])
        edges[parent1[index]].append(parent1[sideIn["right"]])
        
        cIndex = parent2.index(parent1[index])
        sideIn = CalcIndex(parent2, cIndex)
        
        edges[parent1[index]].append(parent2[sideIn["left"]])
        edges[parent1[index]].append(parent2[sideIn["right"]])
        
    return edges

In [ ]:
def FindShortestListKey(lst, edges, child):
    #NOTE: Given that I allow there to be multiple instances of an element in the lists to signify a common edge,
    #          I need to remove the duplicate edges when checking their length
    shortest = []
    size = None
    item = None
    rmDups = None
    
    for key in lst:
        if key not in child: #Might not need anymore
            rmDups = Counter(edges[key])  
            
            if size == None:
                size = len(rmDups)
                shortest.append(key)

            elif len(rmDups) > 0 and len(rmDups) < size:
                size = len(rmDups)
                shortest = [key]

            elif len(rmDups) == size:
                shortest.append(key)

    if len(shortest) > 1:
        item = random.choice(shortest)
                    
    else:
        item = shortest[0]
    
    return item

In [ ]:
#Edge Crossover method of recombination/child generation
def EX(parent1, parent2):
    child = []
    
    #A dictionary with each city as a key that connects to a list of its edges
    edges = None
    
    #Used to create child
    entry = None
    currElement = None
    count = None
    i = 0
    unused = []
    
    #1) Construct Edge Table
    edges = GenerateEdgeTable(parent1, parent2)
    
    #2) Pick a initial element @ random & put in offspring
    entry = random.choice(list(edges))
    child.append(entry)
    
    
    #3) Set the variable current_element = entry
    currElement = entry
    
    while len(child) < len(parent1):
    
        #4) Remove all references to current_element from table
        for key in edges:
            try:
                #NOTE: Should look for a cleaner/better solution
                while True:
                    edges[key].remove(currElement)
                    
            except ValueError:
                pass
        
        #5) Examine list for current_element
        #NOTE: Ties are split at random
        if len(edges[currElement]) != 0:
            count = Counter(edges[currElement]).most_common()
            
            #5.1) If there is a common edge, pick that to be the next element
            if count[0][1] == 2:
                i = 1

                while i < len(count) and count[i][1] == 2:
                    i += 1
                
                entry = random.choice(count[0:i])[0]
                
                while entry in child:
                    entry = random.choice(count[0:i])[0]
                    
                currElement = entry

            #5.2) Otherwise, pick the entry in the list which itself has the shortest list
            else:
                currElement = FindShortestListKey(edges[currElement], edges, child)

        #6) In the case of reaching an empty list, a new element is chosen at random
        else:
            unused = [item for item in parent1 if item not in child]
            
            currElement = random.choice(unused)
        
        #Reset Variables
        i = 0
        unused = []
        
        child.append(currElement)
    
    return child

In [ ]:
#Order Crossover method of recombination/child generation
def OX(parent1, parent2):
    child = [None] * len(parent1)
    notAdded = None
    
    #Choose 2 crossover points at random
    geneA = int(random.random() * len(parent1))
    geneB = int(random.random() * len(parent1))
    
    startGene = min(geneA, geneB)
    endGene = max(geneA, geneB)
    p2Index = endGene
    cpIndex = endGene
    
    #Copy over segment from parent1 into child
    for index in range(startGene, endGene):
        child[index] = parent1[index]
        
    #Start from the end of the crossover point in parent 2 and copy over any elements not currently in child
    notAdded = [item for item in parent2 if item not in child]
    
    for i in range(0, len(notAdded)):
        if parent2[p2Index] not in child:
            if child[cpIndex] != None:
                while child[cpIndex] != None:
                    cpIndex += 1
                    
                    if cpIndex >= len(child):
                        cpIndex = 0
                        
        else:
            while parent2[p2Index] in child:
                p2Index += 1
                    
                if p2Index >= len(parent2):
                    p2Index = 0  
                            
        child[cpIndex] = parent2[p2Index]
                
            
        p2Index += 1
        cpIndex += 1
        
        if p2Index >= len(parent2):
            p2Index = 0
        
        if cpIndex >= len(child):
            cpIndex = 0
        
    return child

In [ ]:
def breedPopulation(matingpool, eliteSize):
    children = []
    length = len(matingpool) - eliteSize
    pool = random.sample(matingpool, len(matingpool))

    for i in range(0,eliteSize):
        children.append(matingpool[i])
    
    for i in range(0, length):
        #child = breed(pool[i], pool[len(matingpool)-i-1])
        child = PMX(pool[i], pool[len(matingpool) - i - 1])
        #child = EX(pool[i], pool[len(matingpool) - i - 1])
        #child = OX(pool[i], pool[len(matingpool) - i - 1])
        children.append(child)
    return children

In [ ]:
def mutate(individual, mutationRate):
    for swapped in range(len(individual)):
        if(random.random() < mutationRate):
            swapWith = int(random.random() * len(individual))
            
            city1 = individual[swapped]
            city2 = individual[swapWith]
            
            individual[swapped] = city2
            individual[swapWith] = city1
    return individual

In [ ]:
def SwapMutation(individual, mutationRate):
    swapped = None
    swapWith = None
    gene1 = None
    gene2 = None
    
    if(random.random() < mutationRate):
        swapped  = random.randint(0, len(individual) - 1)
        swapWith = random.randint(0, len(individual) - 1)
            
        gene1 = individual[swapped]
        gene2 = individual[swapWith]
            
        individual[swapped] = gene2
        individual[swapWith] = gene1
    return individual

In [ ]:
def InversionMutation(individual, mutationRate):
    index1 = None
    index2 = None
    smallIndex = None
    bigIndex = None
    index = 0
    
    gene1 = None
    gene2 = None
    
    if(random.random() < mutationRate):
        #Randomly selects the section of the chromosome to reverse
        index1 = random.randint(0, len(individual) - 1)
        index2 = random.randint(0, len(individual) - 1)

        smallIndex = min(index1, index2)
        bigIndex = max(index1, index2)
        
        swapped = smallIndex + index
        swapWith = bigIndex - index
        
        while swapped < swapWith:
            
            gene1 = individual[swapped]
            gene2 = individual[swapWith]

            individual[swapped] = gene2
            individual[swapWith] = gene1
            
            index += 1
            swapped = smallIndex + index
            swapWith = bigIndex - index
        
    return individual

In [ ]:
def mutatePopulation(population, mutationRate):
    mutatedPop = []
    
    for ind in range(0, len(population)):
        #mutatedInd = mutate(population[ind], mutationRate)
        mutatedInd = InversionMutation(population[ind], mutationRate)
        mutatedPop.append(mutatedInd)
    return mutatedPop

In [ ]:
def nextGeneration(currentGen, eliteSize, mutationRate):
    popRanked = rankRoutes(currentGen)
#    for i in popRanked:
#        print(i)
    selectionResults = selection(popRanked, eliteSize)
    matingpool = matingPool(currentGen, selectionResults)
#    for i in matingpool:
#        print(i)
    children = breedPopulation(matingpool, eliteSize)
    nextGeneration = mutatePopulation(children, mutationRate)
    return nextGeneration

In [ ]:
#def geneticAlgorithm(population, popSize, eliteSize, mutationRate, generations):
def geneticAlgorithm(population, eliteSize, mutationRate, generations):
    #pop = initialPopulation(popSize, population)
    pop = population
    print("Initial distance: " + str(1 / rankRoutes(pop)[0][1]))
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
    
    print("Final distance: " + str(1 / rankRoutes(pop)[0][1]))
    bestRouteIndex = rankRoutes(pop)[0][0]
    bestRoute = pop[bestRouteIndex]
    return bestRoute

In [ ]:
#def geneticAlgorithmPlot(population, popSize, eliteSize, mutationRate, generations):
def geneticAlgorithmPlot(population, eliteSize, mutationRate, generations):
    #pop = initialPopulation(popSize, population)
    pop = population
    progress = []
    progress.append(1 / rankRoutes(pop)[0][1])
    
    for i in range(0, generations):
        pop = nextGeneration(pop, eliteSize, mutationRate)
        progress.append(1 / rankRoutes(pop)[0][1])
    
    plt.plot(progress)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    plt.show()

In [ ]:
def geneticAlgorithmComparisonPlot(population, eliteSize, mutationRate, generations, childMethods, mutationMethods):
    #pop = initialPopulation(popSize, population)
    pop = population
    progress = []
    progress.append(1 / rankRoutes(pop)[0][1])
    label = None
    df = pd.DataFrame()
    
    for cMethod in childMethods:
        for mMethod in mutationMethods:
            label = childMethods[cMethod] + "," + mutationMethods[mMethod]
    
            for i in range(0, generations):
                pop = nextGeneration(pop, eliteSize, mutationRate)
                progress.append(1 / rankRoutes(pop)[0][1])
                
            df[label] = progress
            progress = []
            progress.append(1 / rankRoutes(pop)[0][1])
            pop = population
    
    plt.plot(df)
    plt.ylabel('Distance')
    plt.xlabel('Generation')
    #plt.set_labels(df.columns)
    plt.legend(df)
    plt.show()

In [ ]:
cityList = []

for i in range(0,25):
    cityList.append(City(x=int(random.random() * 200), y=int(random.random() * 200)))

In [ ]:
popSize = 100
eliteSize = 20
mutationRate = 0.01
generations = 500

In [ ]:
childMethods = {0:"None", 1:"PMX", 2:"EX", 3:"OX"}
mutationMethods = {0:"None", 1:"Swap", 2:"Inversion"}

In [ ]:
#Generate a population for testing
pop = initialPopulation(popSize = popSize, cityList = cityList)

In [ ]:
#geneticAlgorithm(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=500)
geneticAlgorithm(population=pop, eliteSize=eliteSize, mutationRate=mutationRate, generations=generations)

In [ ]:
#geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=500)
#geneticAlgorithmPlot(population=pop, eliteSize=20, mutationRate=0.01, generations=500)
geneticAlgorithmComparisonPlot(population=pop, eliteSize=eliteSize, mutationRate=mutationRate, generations=generations,
                               childMethods = childMethods, mutationMethods = mutationMethods)

In [ ]:
#Generate more data for the graph
#geneticAlgorithmPlot(population=cityList, popSize=100, eliteSize=20, mutationRate=0.01, generations=500)
geneticAlgorithmPlot(population=pop, eliteSize=20, mutationRate=0.01, generations=500)

In [ ]:
#The following are for testing purposes

In [ ]:
#Keeps saying that there is no modue named "logProcess"
#from logProcess import log_progress

In [ ]:
#Smaller group for testing the adding of methods
geneticAlgorithmPlot(population=cityList[0:10], popSize=20, eliteSize=5, mutationRate=0.01, generations=20)

In [ ]:
geneticAlgorithm(population=cityList[0:10], popSize=20, eliteSize=5, mutationRate=0.01, generations=20)

In [ ]:
city1 = City(0,1)
city2 = City(1,0)
city3 = City(0,2)
city4 = City(2,0)

In [ ]:
#The following is to test the recombination functions

In [ ]:
numberPop = [i for i in range(0, 25)]

In [ ]:
numberLists = []

for i in range(0, 100):
    numberLists.append(random.sample(numberPop, len(numberPop)))

In [ ]:
children = []

In [ ]:
p1 = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17]
p2 = [2,15,12,5,17,9,14,6,10,16,3,11,7,4,8,1,13]

In [ ]:
p1 = [1,2,3,4,5,6,7,8,9]
p2 = [9,3,7,8,2,6,5,1,4]
p3 = [9,10,11,12,13,14,15,16,17]

In [ ]:
d = {'parent1': p1, 'parent2': p2}
df = pd.DataFrame(data=d)
df

In [ ]:
#df1 = df1.assign(e=pd.Series(np.random.randn(sLength)).values)
#d = {"parent3": p3}
#s = pd.Series(data = d)
#df.assign("parent3"=s.values)
df["parent3"] = p3
df

In [ ]:
plt.plot(df)
plt.ylabel('Distance')
plt.xlabel('Generation')
plt.show()

In [ ]:
df = pd.DataFrame()
df

In [ ]:
df["fish"] = p1
df